# وظیفه طبقه‌بندی متن

در این بخش، با یک وظیفه ساده طبقه‌بندی متن بر اساس مجموعه داده **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** شروع می‌کنیم: تیترهای خبری را به یکی از ۴ دسته زیر طبقه‌بندی خواهیم کرد: جهان، ورزش، تجارت و علم/فناوری.

## مجموعه داده

برای بارگذاری مجموعه داده، از API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** استفاده خواهیم کرد.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

اکنون می‌توانیم با استفاده از `dataset['train']` و `dataset['test']` به بخش‌های آموزش و آزمایش مجموعه داده دسترسی پیدا کنیم:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


بیایید اولین ۱۰ تیتر جدید را از مجموعه داده خود چاپ کنیم:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## بردارسازی متن

حالا باید متن را به **اعداد** تبدیل کنیم تا بتوان آن را به‌صورت تنسورها نمایش داد. اگر بخواهیم متن را در سطح کلمات نمایش دهیم، باید دو کار انجام دهیم:

* استفاده از یک **توکنایزر** برای شکستن متن به **توکن‌ها**.
* ساخت یک **واژگان** از این توکن‌ها.

### محدود کردن اندازه واژگان

در مثال مجموعه داده AG News، اندازه واژگان نسبتاً بزرگ است و بیش از ۱۰۰ هزار کلمه دارد. به‌طور کلی، ما به کلماتی که به‌ندرت در متن ظاهر می‌شوند نیازی نداریم — فقط چند جمله آن‌ها را خواهند داشت و مدل از آن‌ها چیزی یاد نخواهد گرفت. بنابراین، منطقی است که اندازه واژگان را با ارسال یک آرگومان به سازنده وکتورایزر به عدد کوچکتری محدود کنیم:

هر دوی این مراحل را می‌توان با استفاده از لایه **TextVectorization** مدیریت کرد. بیایید شیء وکتورایزر را ایجاد کنیم و سپس با فراخوانی متد `adapt` تمام متن را مرور کرده و یک واژگان بسازیم:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **توجه** ما تنها از یک زیرمجموعه از کل داده‌ها برای ساخت واژگان استفاده می‌کنیم. این کار را برای افزایش سرعت اجرا و جلوگیری از انتظار شما انجام می‌دهیم. با این حال، این خطر وجود دارد که برخی از کلمات موجود در کل مجموعه داده‌ها وارد واژگان نشوند و در طول آموزش نادیده گرفته شوند. بنابراین، استفاده از اندازه کامل واژگان و اجرای آن بر روی کل مجموعه داده در طول `adapt` باید دقت نهایی را افزایش دهد، اما نه به طور قابل توجهی.

اکنون می‌توانیم به واژگان واقعی دسترسی داشته باشیم:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


با استفاده از وکتورایزر، می‌توانیم به‌راحتی هر متنی را به مجموعه‌ای از اعداد کدگذاری کنیم:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## نمایش متن به روش کیسه کلمات

از آنجا که کلمات حامل معنا هستند، گاهی می‌توانیم با نگاه کردن به کلمات جداگانه، بدون توجه به ترتیب آن‌ها در جمله، معنای یک متن را درک کنیم. برای مثال، هنگام دسته‌بندی اخبار، کلماتی مانند *آب‌وهوا* و *برف* احتمالاً به *پیش‌بینی آب‌وهوا* اشاره دارند، در حالی که کلماتی مانند *سهام* و *دلار* به *اخبار مالی* مربوط می‌شوند.

نمایش برداری **کیسه کلمات** (Bag-of-words یا BoW) ساده‌ترین روش سنتی برای نمایش برداری است که می‌توان آن را درک کرد. در این روش، هر کلمه به یک شاخص بردار مرتبط می‌شود و هر عنصر بردار تعداد دفعات وقوع هر کلمه در یک سند مشخص را نشان می‌دهد.

![تصویری که نشان می‌دهد نمایش برداری کیسه کلمات چگونه در حافظه نمایش داده می‌شود.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png)

> **Note**: می‌توانید BoW را به‌عنوان مجموع تمام بردارهای کدگذاری‌شده به روش یک‌داغ (one-hot-encoded) برای کلمات جداگانه در متن نیز در نظر بگیرید.

در زیر یک مثال از نحوه تولید نمایش کیسه کلمات با استفاده از کتابخانه Scikit Learn در پایتون آورده شده است:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

ما همچنین می‌توانیم از بردارساز Keras که در بالا تعریف کردیم استفاده کنیم، هر شماره کلمه را به یک کدگذاری یک‌داغ تبدیل کرده و همه آن بردارها را جمع کنیم:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **توجه**: ممکن است تعجب کنید که نتیجه با مثال قبلی متفاوت است. دلیل این تفاوت این است که در مثال Keras طول بردار با اندازه واژگان مطابقت دارد، که از کل مجموعه داده AG News ساخته شده بود، در حالی که در مثال Scikit Learn ما واژگان را به صورت لحظه‌ای از متن نمونه ساختیم.


## آموزش دسته‌بند BoW

حالا که یاد گرفتیم چگونه نمایش کیسه‌ای از کلمات (bag-of-words) متن خود را بسازیم، بیایید یک دسته‌بند که از این نمایش استفاده می‌کند را آموزش دهیم. ابتدا باید مجموعه داده خود را به نمایش کیسه‌ای از کلمات تبدیل کنیم. این کار را می‌توان با استفاده از تابع `map` به شکل زیر انجام داد:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

حالا بیایید یک شبکه عصبی طبقه‌بندی ساده تعریف کنیم که شامل یک لایه خطی است. اندازه ورودی `vocab_size` است و اندازه خروجی مربوط به تعداد کلاس‌ها (۴) می‌باشد. چون ما در حال حل یک وظیفه طبقه‌بندی هستیم، تابع فعال‌سازی نهایی **softmax** است:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


از آنجایی که ما ۴ کلاس داریم، دقت بالای ۸۰٪ نتیجه‌ی خوبی محسوب می‌شود.

## آموزش یک طبقه‌بند به‌عنوان یک شبکه

از آنجا که وکتورایزر نیز یک لایه‌ی Keras است، می‌توانیم شبکه‌ای تعریف کنیم که آن را شامل شود و به‌صورت انتها به انتها آموزش دهیم. به این ترتیب نیازی به وکتورایز کردن دیتاست با استفاده از `map` نداریم و می‌توانیم دیتاست اصلی را مستقیماً به ورودی شبکه بدهیم.

> **توجه**: همچنان باید از map برای تبدیل فیلدها از دیکشنری‌ها (مانند `title`، `description` و `label`) به تاپل‌ها استفاده کنیم. با این حال، هنگام بارگذاری داده‌ها از دیسک، می‌توانیم از همان ابتدا دیتاستی با ساختار مورد نیاز بسازیم.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## بیگرام‌ها، تریگرام‌ها و اِن‌گرام‌ها

یکی از محدودیت‌های روش کیسه کلمات این است که برخی کلمات بخشی از عبارات چندکلمه‌ای هستند. برای مثال، کلمه «هات داگ» معنای کاملاً متفاوتی نسبت به کلمات «هات» و «داگ» در سایر زمینه‌ها دارد. اگر همیشه کلمات «هات» و «داگ» را با استفاده از بردارهای یکسان نمایش دهیم، ممکن است مدل ما را دچار سردرگمی کند.

برای حل این مشکل، **نمایش‌های اِن‌گرام** اغلب در روش‌های طبقه‌بندی اسناد استفاده می‌شوند، جایی که فراوانی هر کلمه، دوکلمه‌ای یا سه‌کلمه‌ای ویژگی مفیدی برای آموزش طبقه‌بندها است. در نمایش بیگرام، برای مثال، علاوه بر کلمات اصلی، تمام جفت کلمات را نیز به واژگان اضافه می‌کنیم.

در زیر مثالی از نحوه تولید نمایش کیسه کلمات بیگرام با استفاده از Scikit Learn آورده شده است:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

عیب اصلی روش n-gram این است که اندازه واژگان به سرعت بسیار زیاد رشد می‌کند. در عمل، لازم است که نمایش n-gram را با یک تکنیک کاهش ابعاد، مانند *تعبیه‌ها*، ترکیب کنیم که در واحد بعدی درباره آن صحبت خواهیم کرد.

برای استفاده از نمایش n-gram در مجموعه داده **AG News**، باید پارامتر `ngrams` را به سازنده `TextVectorization` خود منتقل کنیم. طول واژگان بی‌گرام **به طور قابل توجهی بزرگ‌تر** است، در مورد ما بیش از 1.3 میلیون توکن! بنابراین منطقی است که تعداد توکن‌های بی‌گرام را نیز به یک عدد معقول محدود کنیم.

می‌توانیم از همان کدی که در بالا استفاده کردیم برای آموزش طبقه‌بند استفاده کنیم، اما این کار از نظر حافظه بسیار ناکارآمد خواهد بود. در واحد بعدی، طبقه‌بند بی‌گرام را با استفاده از تعبیه‌ها آموزش خواهیم داد. در همین حال، می‌توانید با آموزش طبقه‌بند بی‌گرام در این دفترچه آزمایش کنید و ببینید آیا می‌توانید دقت بالاتری به دست آورید.


## محاسبه خودکار بردارهای BoW

در مثال بالا، بردارهای BoW را به صورت دستی با جمع کردن کدگذاری‌های یک‌داغ کلمات جداگانه محاسبه کردیم. با این حال، نسخه جدید TensorFlow به ما این امکان را می‌دهد که با استفاده از پارامتر `output_mode='count` در سازنده وکتورایزر، بردارهای BoW را به صورت خودکار محاسبه کنیم. این کار تعریف و آموزش مدل ما را به طور قابل توجهی ساده‌تر می‌کند:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## فراوانی واژه - فراوانی معکوس سند (TF-IDF)

در نمایش BoW، وقوع واژه‌ها با استفاده از یک تکنیک یکسان وزن‌دهی می‌شود، بدون توجه به خود واژه. با این حال، واضح است که واژه‌های پرتکراری مانند *a* و *in* برای طبقه‌بندی بسیار کم‌اهمیت‌تر از اصطلاحات تخصصی هستند. در بیشتر وظایف پردازش زبان طبیعی (NLP)، برخی واژه‌ها نسبت به دیگران مرتبط‌تر هستند.

**TF-IDF** مخفف **فراوانی واژه - فراوانی معکوس سند** است. این یک نوع تغییر یافته از مدل کیسه واژه‌ها (bag-of-words) است که به جای استفاده از مقدار دودویی ۰/۱ برای نشان دادن حضور یک واژه در یک سند، از یک مقدار اعشاری استفاده می‌کند که به فراوانی وقوع واژه در مجموعه متون مرتبط است.

به طور رسمی‌تر، وزن $w_{ij}$ یک واژه $i$ در سند $j$ به صورت زیر تعریف می‌شود:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
که در آن:
* $tf_{ij}$ تعداد وقوع‌های $i$ در $j$ است، یعنی همان مقدار BoW که قبلاً دیده‌ایم
* $N$ تعداد کل اسناد در مجموعه است
* $df_i$ تعداد اسنادی است که واژه $i$ را در کل مجموعه شامل می‌شوند

مقدار TF-IDF یعنی $w_{ij}$ به طور متناسب با تعداد دفعاتی که یک واژه در یک سند ظاهر می‌شود افزایش می‌یابد و با تعداد اسناد در مجموعه که آن واژه را شامل می‌شوند تنظیم می‌شود. این امر کمک می‌کند تا برای این واقعیت که برخی واژه‌ها بیشتر از دیگران ظاهر می‌شوند، تعدیل صورت گیرد. برای مثال، اگر یک واژه در *تمامی* اسناد مجموعه ظاهر شود، $df_i=N$ و $w_{ij}=0$ خواهد بود و آن واژه‌ها به طور کامل نادیده گرفته می‌شوند.

شما می‌توانید به راحتی بردارسازی TF-IDF متن را با استفاده از Scikit Learn ایجاد کنید:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

در کراس، لایه `TextVectorization` می‌تواند با استفاده از پارامتر `output_mode='tf-idf'` به طور خودکار فراوانی‌های TF-IDF را محاسبه کند. بیایید کدی که در بالا استفاده کردیم را تکرار کنیم تا ببینیم آیا استفاده از TF-IDF دقت را افزایش می‌دهد:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## نتیجه‌گیری

با اینکه نمایش‌های TF-IDF وزن‌های فرکانسی را به کلمات مختلف اختصاص می‌دهند، اما قادر به نمایش معنا یا ترتیب نیستند. همان‌طور که زبان‌شناس مشهور جی. آر. فرث در سال 1935 گفت: "معنای کامل یک کلمه همیشه وابسته به متن است و هیچ مطالعه‌ای درباره معنا بدون در نظر گرفتن متن نمی‌تواند جدی گرفته شود." در ادامه دوره، یاد خواهیم گرفت که چگونه اطلاعات متنی را با استفاده از مدل‌سازی زبان استخراج کنیم.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
